In [2]:
import pandas as pd
import psycopg2
import numpy as np
import os

from dotenv import load_dotenv
   
from sqlalchemy import create_engine

# load up the variables from the .env file.
load_dotenv()

True

### States Abbreviation Table

In [3]:
# File to Load (Remember to Change These)
file_to_load = "Resources/StatetoAbbrev.csv"
states_data = pd.read_csv(file_to_load)
states_data.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [4]:
states_data['State_id']=states_data.index+1
states_data.head()

,State,Abbrev,Code,State_id
0,Alabama,Ala.,AL,1
1,Alaska,Alaska,AK,2
2,Arizona,Ariz.,AZ,3
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,5


In [5]:
len(states_data)

51

### Natural Gas Production Table

In [6]:
file_to_load = "Resources/Natural Gas Marketed Production-StateRankings.csv"

natural_gas_production = pd.read_csv(file_to_load)
natural_gas_production.head()

,Rank,State,"Natural Gas Marketed Production, million cu ft",Note: Rankings are based on the full source data values. Excludes Gulf of Mexico federal offshore production.
0,1,TX,7847102,NaN
1,2,PA,6210673,NaN
2,3,OK,2946117,NaN
3,4,LA,2810636,NaN
4,5,OH,2409153,NaN


#### Clean the 'State' column

Some rows have full state name instead of state code.

In [7]:
# Set 'State' as index in State Abb table
state_df = states_data.set_index('State')

# Changing full name to state code
for i in natural_gas_production.index:
    s = natural_gas_production.loc[i,'State']
    if len(s)>2:
        natural_gas_production.loc[i,'State'] = state_df.loc[s,'Code']

In [8]:
naturalgas_df = pd.merge(states_data, natural_gas_production, how='outer', left_on='Code', right_on='State')

naturalgas_df.drop(columns = ["State_x","State_y","Code","Abbrev","Note: Rankings are based on the full source data values. Excludes Gulf of Mexico federal offshore production."], inplace = True)

naturalgas_df.head()

,State_id,Rank,"Natural Gas Marketed Production, million cu ft"
0,1,16,139485
1,2,12,341315
2,3,31,46
3,4,11,589973
4,5,14,202616


In [9]:
for cl in naturalgas_df.columns:
    print(cl)

State_id
Rank
Natural Gas Marketed Production, million cu ft


### Energy Production Estimate Table

In [10]:
file_to_load = "Resources/P2.csv"
production_df = pd.read_csv(file_to_load)

production_df.rename(columns ={"Table P2.  Primary Energy Production Estimates in Trillion Btu, 2018 ":"State",
                               "Unnamed: 1":"Fossil Fuels", "Unnamed: 3":"Natural Gas", "Unnamed: 5":"Crude Oil",
                              "Unnamed: 9":"Biofuels", "Unnamed: 7":"Nuclear Electric Power","Unnamed: 11":"Wood and Waste",
                              "Unnamed: 13":"Other", "Unnamed: 15":"Total"}, inplace = True)

production_df.drop(columns = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8','Unnamed: 10', 'Unnamed: 12', 'Unnamed: 14',
                             'Unnamed: 16'], inplace = True)

production_df.drop([0,1,2,3], inplace = True)
production_df.head()

,State,Fossil Fuels,Natural Gas,Crude Oil,Nuclear Electric Power,Biofuels,Wood and Waste,Other,Total
4,Alabama,370.5,149.6,33.6,412.6,1.7,170.1,105.0,"1,243.0"
5,Alaska,13.8,375.3,997.4,0.0,(s),7.3,16.8,"1,410.6"
6,Arizona,140.8,(s),0.1,325.1,6.8,8.7,139.5,621.0
7,Arkansas,0.0,600.4,28.6,133.0,9.5,85.2,30.3,887.1
8,California,0.0,228.9,965.3,190.4,35.5,130.5,857.6,"2,408.2"


In [11]:
production_df['State'].values

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', nan,
       'Federal Offshore - Gulf of Mexico', 'Federal Offshore - Pacific',
       nan, 'United States', nan, nan, 'a  Includes refuse recovery.',
       'b  Marketed production.', 'c  Includes lease condensate.',
       'd  Biomass inputs (feedstock) to the production of b

In [12]:
# Merge the rows only the state names exist : left(state_data) merge
productionfinal_df = pd.merge(states_data,production_df,how='left', left_on='State', right_on='State')
productionfinal_df.drop(columns=["Abbrev", "State", "Code"], inplace = True)
productionfinal_df.head()

,State_id,Fossil Fuels,Natural Gas,Crude Oil,Nuclear Electric Power,Biofuels,Wood and Waste,Other,Total
0,1,370.5,149.6,33.6,412.6,1.7,170.1,105.0,"1,243.0"
1,2,13.8,375.3,997.4,0.0,(s),7.3,16.8,"1,410.6"
2,3,140.8,(s),0.1,325.1,6.8,8.7,139.5,621.0
3,4,0.0,600.4,28.6,133.0,9.5,85.2,30.3,887.1
4,5,0.0,228.9,965.3,190.4,35.5,130.5,857.6,"2,408.2"


In [13]:
# 
#productionfinal_df = pd.merge(states_data,production_df,how='outer', left_on='State', right_on='State')
#productionfinal_df.drop(columns=["Abbrev", "State", "Code"], inplace = True)
#productionfinal_df.head()

In [14]:
productionfinal_df.columns

Index(['State_id', 'Fossil Fuels', 'Natural Gas', 'Crude Oil',
       'Nuclear Electric Power', 'Biofuels', 'Wood and Waste', 'Other',
       'Total'],
      dtype='object')

### Renewable Potential energy

In [15]:
# File to Load (Remember to Change These)
file_to_load = "Resources/usretechnicalpotential.csv"

# Read Renewable energy potential file and store into Pandas data frame
renewable_energy_data = pd.read_csv(file_to_load)
renewable_energy_data.columns

Index(['Unnamed: 0', 'urbanUtilityScalePV_GWh', 'urbanUtilityScalePV_GW',
       'urbanUtilityScalePV_km2', 'ruralUtilityScalePV_GWh',
       'ruralUtilityScalePV_GW', 'ruralUtilityScalePV_km2', 'rooftopPV_GWh',
       'rooftopPV_GW', 'CSP_GWh', 'CSP_GW', 'CSP_km2', 'onshoreWind_GWh',
       'onshoreWind_GW', 'onshoreWind_km2', 'offshoreWind_GWh',
       'offshoreWind_GW', 'offshoreWind_km2', 'biopowerSolid_GWh',
       'biopowerSolid_GW', 'biopowerSolid_BDT', 'biopowerGaseous_GWh',
       'biopowerGaseous_GW', 'biopowerGaseous_Tonnes-CH4',
       'geothermalHydrothermal_GWh', 'geothermalHydrothermal_GW',
       'EGSGeothermal_GWh', 'EGSGeothermal_GW', 'hydropower_GWh',
       'hydropower_GW', 'hydropower_countOfSites'],
      dtype='object')

In [16]:
# Rename unnamed column as State
renewable_energy_data.rename(columns={'Unnamed: 0' : 'State'}, inplace=True)

# Select columns
renewable_potential = renewable_energy_data[[
                        'State',
                        'urbanUtilityScalePV_GWh', 'ruralUtilityScalePV_GWh',
                        'rooftopPV_GWh', 'CSP_GWh',
                        'onshoreWind_GWh', 'offshoreWind_GWh',                        
                        'biopowerSolid_GWh', 'biopowerGaseous_GWh',
                        'geothermalHydrothermal_GWh',
                        'EGSGeothermal_GWh',
                        'hydropower_GWh']]
renewable_potential.head()

,State,urbanUtilityScalePV_GWh,ruralUtilityScalePV_GWh,rooftopPV_GWh,CSP_GWh,onshoreWind_GWh,offshoreWind_GWh,biopowerSolid_GWh,biopowerGaseous_GWh,geothermalHydrothermal_GWh,EGSGeothermal_GWh,hydropower_GWh
0,Alabama,35850,3706838,15475.0,0,283,0.0,11193,1533,0,535489.0,4102
1,Alaska,166,8282976,NaN,0,1373433,NaN,513,61,15437,NaN,23675
2,Arizona,121305,11867693,22736.0,12544333,26036,NaN,1087,837,8329,1239147.0,1303
3,Arkansas,28960,4986388,8484.0,0,22892,NaN,14381,1063,0,628621.0,6093
4,California,246008,8855917,106411.0,8490916,89862,2662579.0,12408,15510,130921,1344179.0,30023


In [17]:
# To set State_id, merge this table with State Abb table
combined_df = pd.merge(states_data,renewable_potential,how='outer', on='State')
combined_df.head()

,State,Abbrev,Code,State_id,urbanUtilityScalePV_GWh,ruralUtilityScalePV_GWh,rooftopPV_GWh,CSP_GWh,onshoreWind_GWh,offshoreWind_GWh,biopowerSolid_GWh,biopowerGaseous_GWh,geothermalHydrothermal_GWh,EGSGeothermal_GWh,hydropower_GWh
0,Alabama,Ala.,AL,1,35850,3706838,15475.0,0,283,0.0,11193,1533,0,535489.0,4102
1,Alaska,Alaska,AK,2,166,8282976,NaN,0,1373433,NaN,513,61,15437,NaN,23675
2,Arizona,Ariz.,AZ,3,121305,11867693,22736.0,12544333,26036,NaN,1087,837,8329,1239147.0,1303
3,Arkansas,Ark.,AR,4,28960,4986388,8484.0,0,22892,NaN,14381,1063,0,628621.0,6093
4,California,Calif.,CA,5,246008,8855917,106411.0,8490916,89862,2662579.0,12408,15510,130921,1344179.0,30023


In [18]:
m = len(combined_df.index)
print(m)

51


In [19]:
renewable_df = combined_df.drop(columns=['State', 'Abbrev', 'Code'])
#renewable_df.set_index('State_id', inplace=True)
renewable_df.head()

,State_id,urbanUtilityScalePV_GWh,ruralUtilityScalePV_GWh,rooftopPV_GWh,CSP_GWh,onshoreWind_GWh,offshoreWind_GWh,biopowerSolid_GWh,biopowerGaseous_GWh,geothermalHydrothermal_GWh,EGSGeothermal_GWh,hydropower_GWh
0,1,35850,3706838,15475.0,0,283,0.0,11193,1533,0,535489.0,4102
1,2,166,8282976,NaN,0,1373433,NaN,513,61,15437,NaN,23675
2,3,121305,11867693,22736.0,12544333,26036,NaN,1087,837,8329,1239147.0,1303
3,4,28960,4986388,8484.0,0,22892,NaN,14381,1063,0,628621.0,6093
4,5,246008,8855917,106411.0,8490916,89862,2662579.0,12408,15510,130921,1344179.0,30023


### Electricity Production Table

In [20]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Total Net Electricity Generation-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
total_electricity_data = pd.read_csv(file_to_load)
total_electricity_data.head()

,Rank,State,"Total Net Electricity Generation, thousand MWh",Note: Rankings are based on the full source data values.
0,1,TX,38524,NaN
1,2,FL,20787,NaN
2,3,PA,16521,NaN
3,4,CA,15465,NaN
4,5,IL,12997,NaN


In [21]:
total_electricity = total_electricity_data.drop(columns='Note: Rankings are based on the full source data values.')
total_electricity.rename(columns={'Total Net Electricity Generation, thousand MWh': 'Total_Net_GWh'}, inplace=True)
total_electricity.head()

,Rank,State,Total_Net_GWh
0,1,TX,38524
1,2,FL,20787
2,3,PA,16521
3,4,CA,15465
4,5,IL,12997


In [22]:
combined_df = pd.merge(states_data,total_electricity,how='outer', left_on='Code', right_on='State')
combined_df.head()

,State_x,Abbrev,Code,State_id,Rank,State_y,Total_Net_GWh
0,Alabama,Ala.,AL,1,7,AL,10575
1,Alaska,Alaska,AK,2,48,AK,476
2,Arizona,Ariz.,AZ,3,10,AZ,9350
3,Arkansas,Ark.,AR,4,31,AR,3532
4,California,Calif.,CA,5,4,CA,15465


In [23]:
total_electricity_df = combined_df[['State_id', 'Rank', 'Total_Net_GWh']]
#total_electricity_df.set_index('State_id', inplace=True)
total_electricity_df.head()

,State_id,Rank,Total_Net_GWh
0,1,7,10575
1,2,48,476
2,3,10,9350
3,4,31,3532
4,5,4,15465


### Coal and Crude Oil Produtcion Table

In [24]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Coal Production-StateRankings.csv"
Coal_Production = pd.read_csv(file_to_load)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",Note: Rankings are based on the full source data values.
0,1,WY,304188,NaN
1,2,WV,95365,NaN
2,3,PA,49883,NaN
3,4,IL,49563,NaN
4,5,KY,39567,NaN


In [25]:
Coal_Production.drop(columns=["Note: Rankings are based on the full source data values."],inplace=True)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons"
0,1,WY,304188
1,2,WV,95365
2,3,PA,49883
3,4,IL,49563
4,5,KY,39567


In [26]:
Coal_combined_df = pd.merge(states_data,Coal_Production,how='inner', left_on='Code', right_on='State')
Coal_combined_df.head()

,State_x,Abbrev,Code,State_id,Rank,State_y,"Coal Production, thousand short tons"
0,Alabama,Ala.,AL,1,10,AL,14783
1,Alaska,Alaska,AK,2,20,AK,902
2,Arizona,Ariz.,AZ,3,16,AZ,6550
3,Colorado,Colo.,CO,6,11,CO,14026
4,Illinois,Ill.,IL,14,4,IL,49563


In [27]:
Coal_combined_df.drop(columns=["State_x", "State_y", "Abbrev", "Code"],inplace=True)
Coal_combined_df.head()

,State_id,Rank,"Coal Production, thousand short tons"
0,1,10,14783
1,2,20,902
2,3,16,6550
3,6,11,14026
4,14,4,49563


In [28]:
# Print the columns
Coal_combined_df.columns

Index(['State_id', 'Rank', 'Coal Production, thousand short tons'], dtype='object')

### Crude Oil Production Table

In [29]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Crude Oil Production-StateRankings.csv"
CrudeOil_Production = pd.read_csv(file_to_load)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,4395,NaN
1,2,NM,885,NaN
2,3,ND,862,NaN
3,4,CO,471,NaN
4,5,AK,404,NaN


In [30]:
CrudeOil_Production.drop(columns=["Note: Rankings are based on the full source data values. Excludes federal offshore production."],inplace=True)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day"
0,1,TX,4395
1,2,NM,885
2,3,ND,862
3,4,CO,471
4,5,AK,404


In [31]:
Crude_combined_df = pd.merge(states_data,CrudeOil_Production,how='inner', left_on='Code', right_on='State')
Crude_combined_df.head()

,State_x,Abbrev,Code,State_id,Rank,State_y,"Crude Oil Production, thousand barrels per day"
0,Alabama,Ala.,AL,1,20,AL,6
1,Alaska,Alaska,AK,2,5,AK,404
2,Arizona,Ariz.,AZ,3,31,AZ,0
3,Arkansas,Ark.,AR,4,18,AR,10
4,California,Calif.,CA,5,6,CA,392


In [32]:
Crude_combined_df.drop(columns=["State_x", "State_y", "Abbrev", "Code"],inplace=True)
Crude_combined_df.head()

,State_id,Rank,"Crude Oil Production, thousand barrels per day"
0,1,20,6
1,2,5,404
2,3,31,0
3,4,18,10
4,5,6,392


In [33]:
Crude_combined_df.columns

Index(['State_id', 'Rank', 'Crude Oil Production, thousand barrels per day'], dtype='object')

### Total Energy Consumed Table

In [34]:
# Load up the CSV file for energy consumption
file_to_load = "Resources/Total Energy Consumed per Capita-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
Totalenergyconsumed = pd.read_csv(file_to_load)
Totalenergyconsumed.head()

,Rank,State,"Total Energy Consumed per Capita, million Btu",Note: Rankings are based on the full source data values.
0,1,WY,967,NaN
1,2,LA,945,NaN
2,3,ND,872,NaN
3,4,AK,830,NaN
4,5,IA,513,NaN


In [35]:
# drop the last column by only pulling over the first 3
Totalenergyconsumed = Totalenergyconsumed[['Rank', 'State', 'Total Energy Consumed per Capita, million Btu']]

# Do an inner join to make sure we only have complete data and pull in the state_id.  This is done comparing the code field 
# the States data table and the State field from the main table.  This will allow all the tables to have the same access State_id 
# versus either complete state name or the state abbreviation.
Totalenergycon_clean = pd.merge(Totalenergyconsumed, states_data,how='inner',left_on='State', right_on='Code')

# Drop the extra columns from the merge and only keep the state_id.
Totalenergycon_clean = Totalenergycon_clean[['State_id','Rank','Total Energy Consumed per Capita, million Btu']]

#display the final table.
Totalenergycon_clean.head()

,State_id,Rank,"Total Energy Consumed per Capita, million Btu"
0,51,1,967
1,19,2,945
2,35,3,872
3,2,4,830
4,16,5,513


### Total Energy Production Table

In [36]:
# Load up the CSV file for energy production
file_to_load = "Resources/Total Energy Production-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
Totalenergyproduction = pd.read_csv(file_to_load)
Totalenergyproduction.head()

,Rank,State,"Total Energy Production, trillion Btu",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,20421,NaN
1,2,PA,8987,NaN
2,3,WY,7718,NaN
3,4,OK,4881,NaN
4,5,WV,4770,NaN


In [37]:
# drop the last column by only pulling over the first 3
Totalenergyproduction = Totalenergyproduction[['Rank', 'State', 'Total Energy Production, trillion Btu']]

# Do an inner join to make sure we only have complete data and pull in the state_id.  This is done comparing the code field 
# the States data table and the State field from the main table.  This will allow all the tables to have the same access State_id 
# versus either complete state name or the state abbreviation.
Totalenergyprod_clean = pd.merge(Totalenergyproduction, states_data,how='inner',left_on='State', right_on='Code')

# Drop the extra columns from the merge and only keep the state_id.
Totalenergyprod_clean = Totalenergyprod_clean[['State_id','Rank','Total Energy Production, trillion Btu']]

#display the final table.
Totalenergyprod_clean.head()

,State_id,Rank,"Total Energy Production, trillion Btu"
0,44,1,20421
1,39,2,8987
2,51,3,7718
3,37,4,4881
4,49,5,4770


## Loading the data into SQL DB

In [38]:
# set the variables for the pull from SQL.  username and password are in an .env file.  You will need those to make this run
# on your machine.
host = 'localhost'
port = 5432
dbname='ETL_Project'
username = os.environ.get('USER_NAME')
pwd = os.environ.get('PASSWORD')

In [39]:
# set up the connection to the database "ETL_Project"
connection_string = f'{username}:{pwd}@{host}:{port}/{dbname}'
engine = create_engine(f'postgresql://{connection_string}')

In [40]:
engine.table_names()

['state_abb',
 'renewable_energy_potential',
 'electricity_production',
 'total_energy_consumed',
 'total_energy_production']

EJ

In [41]:
# Load 'State Abb' table
states_data.to_sql(name='State_Abb', con=engine, if_exists='append', index=False)

In [42]:
# Load 'Renewable_Energy_Potential' table
renewable_df.to_sql(name='Renewable_Energy_Potential', con=engine, if_exists='append', index=False)

In [43]:
# Load 'Total Net Electricity' table
total_electricity_df.to_sql(name='Electricity_Production', con=engine, if_exists='append', index=False)

In [44]:
# Confirm data has been added by querying the State_Abb table
pd.read_sql_query('select "State_id", "State"  from "State_Abb"', con=engine).head()

,State_id,State
0,1,Alabama
1,2,Alaska
2,3,Arizona
3,4,Arkansas
4,5,California


In [45]:
# Confirm data has been added by querying the Renewable_Energy_Potential table
pd.read_sql_query('select * from "Renewable_Energy_Potential"', con=engine).head()

,State_id,urbanUtilityScalePV_GWh,ruralUtilityScalePV_GWh,rooftopPV_GWh,CSP_GWh,onshoreWind_GWh,offshoreWind_GWh,biopowerSolid_GWh,biopowerGaseous_GWh,geothermalHydrothermal_GWh,EGSGeothermal_GWh,hydropower_GWh
0,1,35850,3706838,15475.0,0,283,0.0,11193,1533,0,535489.0,4102
1,2,166,8282976,NaN,0,1373433,NaN,513,61,15437,NaN,23675
2,3,121305,11867693,22736.0,12544333,26036,NaN,1087,837,8329,1239147.0,1303
3,4,28960,4986388,8484.0,0,22892,NaN,14381,1063,0,628621.0,6093
4,5,246008,8855917,106411.0,8490916,89862,2662579.0,12408,15510,130921,1344179.0,30023


Niral

Teshanee

Bill

In [46]:
# Load 'Total_Energy_Production' table
Totalenergyprod_clean.to_sql(name='Total_Energy_Production', con=engine, if_exists='append', index=False)

In [47]:
# Load 'Total_Energy_Consumed' table
Totalenergycon_clean.to_sql(name='Total_Energy_Consumed', con=engine, if_exists='append', index=False)

In [48]:
# Confirm data has been added by querying the State_Abb table
pd.read_sql_query('select *  from "Total_Energy_Production"', con=engine).head()

,State_id,Rank,"Total Energy Production, trillion Btu"
0,44,1,20421
1,39,2,8987
2,51,3,7718
3,37,4,4881
4,49,5,4770


In [49]:
# Confirm data has been added by querying the Total_Energy_Consumed table
pd.read_sql_query('select *  from "Total_Energy_Consumed"', con=engine).head()

,State_id,Rank,"Total Energy Consumed per Capita, million Btu"
0,51,1,967
1,19,2,945
2,35,3,872
3,2,4,830
4,16,5,513


In [50]:
pd.read_sql_query('select * from "Total_Energy_Consumed" tec inner join "State_Abb" sa on tec."State_id"=sa."State_id"', con=engine).head()

,State_id,Rank,"Total Energy Consumed per Capita, million Btu",State,Abbrev,Code,State_id
0,51,1,967,Wyoming,Wyo.,WY,51
1,19,2,945,Louisiana,La.,LA,19
2,35,3,872,North Dakota,N.D.,ND,35
3,2,4,830,Alaska,Alaska,AK,2
4,16,5,513,Iowa,Iowa,IA,16
